<a href="https://colab.research.google.com/github/mpnsk/ivy_seminar/blob/main/ivy_mnist_from_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ivy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net()
import ivy
arg = ivy.random_normal(shape=(64,1,28,28))
transpiled = ivy.transpile(model, source="torch", to="ivy", args=(arg,))


In [ ]:
import torch
from torchvision import datasets, transforms

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('./data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('./data', train=False, transform=transform)
train_kwargs = {'batch_size': 64}

train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
# test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

import ivy
data, target = next(iter(train_loader))
print(f"{data.shape=}")
numpy_array = data.numpy()

backend = "tensorflow"
ivy.set_backend(backend)
ivy_array = ivy.array(numpy_array)
type(ivy_array.data)

# conv = ivy.Conv2D(1,32, 3, 1, 'VALID')
# model = ivy.Sequential(
#     conv
# )



output = transpiled(ivy_array)
print(f"{output.shape=}")
output

data.shape=torch.Size([64, 1, 28, 28])
output.shape=ivy.Shape(64, 10)


/usr/local/lib/python3.10/dist-packages/ivy/func_wrapper.py:242: UserWarning: Creating many views will lead to overhead when performing inplace updates with this backend
  warnings.warn(


ivy.array([[-2.73987532, -2.33122706, -2.50848937, -2.1388166 , -2.21531463,
        -2.18260598, -2.30897069, -2.26526833, -2.26546216, -2.20504451],
       [-2.58528137, -2.35929918, -2.53843927, -2.2009759 , -2.27956295,
        -2.26310945, -2.15949464, -2.23536682, -2.34078217, -2.15839863],
       [-2.42282295, -2.35718799, -2.45274758, -2.23084617, -2.42166233,
        -2.30214119, -2.28517985, -2.15734315, -2.34536076, -2.11089158],
       [-2.49564052, -2.3479023 , -2.41727543, -2.20541215, -2.24456763,
        -2.22854614, -2.31107688, -2.24997735, -2.32378221, -2.23960352],
       [-2.51228714, -2.38684893, -2.37595391, -2.16655993, -2.29562616,
        -2.33853602, -2.23439527, -2.2328682 , -2.32904053, -2.20121932],
       [-2.40195298, -2.31938171, -2.43330431, -2.1582725 , -2.42043543,
        -2.31865048, -2.28762937, -2.2111423 , -2.3507781 , -2.16991377],
       [-2.3655405 , -2.35364032, -2.35479093, -2.18107152, -2.3423903 ,
        -2.23341966, -2.24400663, -2.3956